# TTRPG Chatbot

Really this bot can easily be made into anything by changing the prompt slightly.  It works with pinecone to keep track of the conversation as it happens and retrieve the most useful bits of the conversation for the bot.  Note that I am saving a good amount (including the vector) to the json logs saved in the logs directory.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
import time
import datetime
import json
import re

import pinecone
import openai

openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

embedding_dimensions = 1536
model_engine = "text-embedding-ada-002"
pinecone_name = "dnd-rules-lawyer"
pinecone_region = "asia-southeast1-gcp" # Pinecone calls this the environement ? strange

pinecone.init(api_key=pinecone_api_key, environment=pinecone_region)
index = pinecone.Index(pinecone_name)
openai.api_key = openai_api_key

Below are the helper functions for using the chatbot.

Beware the commented code.  It will delete your vector database.

In [1]:
#results = index.query(vector=[0 for i in range(1536)], top_k=1000)
#print(results)
#index.delete([x["id"] for x in results["matches"]])

In [ ]:
def create_metadata(content, user, vector):
    create_time = time.time()
    return {
        "Timestamp": create_time,
        "User": "User",
        "Message": content,
        "Vector": vector
    }

In [ ]:
def create_message_hash(content):
    return str(hash(content))

In [ ]:
def get_openai_embeddings(content, engine="text-embedding-ada-002"):
    content = content.encode(encoding="ASCII", errors="ignore").decode()  # fix unicode errors
    response = openai.Embedding.create(input=content, engine=engine)
    vector = response['data'][0]['embedding']
    return vector

In [ ]:
def openai_completion(prompt, model="text-davinci-003", temperature=0, top_p=1.0, max_tokens=400, freq_pen=0.0, pres_pen=0.0, max_retry=5):
        retry = 0
        prompt = prompt.encode(encoding="ASCII", errors="ignore").decode()
        while True:
            try:
                response = openai.Completion.create(
                    model=model,
                    prompt=prompt,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=top_p,
                    frequency_penalty=freq_pen,
                    presence_penalty=pres_pen
                )
                text = response["choices"][0]["text"].strip()
                text = re.sub(r'[\r\n]+', "\n", text)
                text = re.sub(r'[\t]+', " ", text)
                return text
                    
            except Exception as e:
                retry += 1
                if retry >= max_retry:
                    return f"GPT3 error: {e}"
                print("Error in communication with openai.")
                time.sleep(1)

In [ ]:
def load_json(filepath):
    with open(filepath, "r", encoding="utf-8") as open_file:
        return json.load(open_file)

In [ ]:
def load_conversation(results):
    result = list()
    for m in results["matches"]:
        if m['id'] == "GPT3 error: Unrecognized request argument supplied: stops": # For some reason pinecone will not remove this from the database
            continue
        info = load_json(f'./logs/{m["id"]}.json')
        result.append(info)
    ordered = sorted(result, key=lambda d: d['Timestamp'], reverse=False)
    messages = [i["Message"] for i in ordered]
    return "\n".join(messages).strip()

In [ ]:
def get_prompt(file_name):
    with open(file_name) as open_file:
        return open_file.read()

In [ ]:
def save_json(filepath, content):
    with open(filepath + ".json", "w+") as f:
        json.dump(content, f)

# Prompting

Here is the interesting part of the bot.  This prompt below can be changed to fit your needs and used to create chatbots that have l

In [ ]:
get_prompt("prompt.txt")

In [ ]:
user = "USER"
top_k = 15
prompt_file = "prompt.txt"
logs_path = "./logs/"

print("Welcome to the TTRPG chatbot.  Ask a ruling here!")
while True:
    payload = list()
    
    # Message Meta
    message = input("\n\n USER: ")
    message_hash = create_message_hash(message)

    # Create embedding of new message
    message_vector = get_openai_embeddings(message)
    
    # Save metadata about vector
    metadata = create_metadata(message, user, message_vector)
    save_json(logs_path + str(message_hash), metadata)
    
    # Append to payload for later indexing
    # Send to Pinecone after gpt message
    payload.append((message_hash, message_vector))
    
    # Query for relevant messages, generate response
    results = index.query(vector=message_vector, top_k=top_k)
    conversation = load_conversation(results)
    prompt = get_prompt(prompt_file).replace("<PREVIOUS_CONVERSATION>", conversation).replace("<MESSAGE>", message)
    
    # Generate the response from the large lang model
    output = openai_completion(prompt)
    output_hash = create_message_hash(output)
    
    # Embed the output
    output_vector = get_openai_embeddings(output)

    # Save the output metadata
    metadata = create_metadata(output, "The Rule Lich", output_vector)
    save_json(logs_path + str(output_hash), metadata)
    
    # Append to the payload the response from gpt
    payload.append((output_hash, output_vector))
    
    # Upsert to the pinecone database
    index.upsert(payload)
    
    # Print responce to the message
    print(f'\n {output}')
    